In [55]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16

In [56]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [57]:
import os
os.chdir('/content/MyDrive/MyDrive/Cat And Dog')

In [58]:
from keras.preprocessing.image import ImageDataGenerator

In [59]:
train_aumentaion=ImageDataGenerator(
    rescale=.1/255,
    rotation_range=40,
    height_shift_range=0.2,
    width_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [60]:
train_data=train_aumentaion.flow_from_directory(
    directory='./training_set',
    target_size=(128,128),
    batch_size=128,
    class_mode='binary'
)

Found 5093 images belonging to 2 classes.


In [61]:
test_restrication=ImageDataGenerator(
    rescale=.1/255
)

In [62]:
test_data=test_restrication.flow_from_directory(
    directory='./test_set',
    target_size=(128,128),
    batch_size=128,
    class_mode='binary'
)

Found 2033 images belonging to 2 classes.


In [63]:
vegg_base=VGG16(input_shape=(128,128,3),include_top=False,weights='imagenet')

In [64]:
vegg_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [65]:
for layer in vegg_base.layers:
  layer.trainable=False

In [75]:
from keras.layers import Dense,Flatten
from keras.losses import binary_crossentropy

In [76]:
model=Sequential()
model.add(vegg_base)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss=binary_crossentropy,metrics=['acc'])

In [77]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense_14 (Dense)            (None, 128)               1048704   
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 15763521 (60.13 MB)
Trainable params: 1048833 (4.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model.fit(train_data,steps_per_epoch=8,epochs=5,validation_data=test_data)

Epoch 1/5
8/8 [==============================] - ETA: 0s - loss: 1.3634 - acc: 0.5557 